[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a DIY Covid Dashboard, it shows a comparison of the **number of new cases** with the **number of tests**.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

Loading local data in ```defData.json``` file.

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("defData.json","rt") as INFILE:
    jsondata=json.load(INFILE)

## Wrangle the data

Wrangling the raw data into a ```DataFrame```.

In [4]:

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object"""
    return pd.to_datetime(datestring, format = "%Y-%m-%d")

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist = rawdata['data']
    dates = [ dictionary['date'] for dictionary in datalist ]
    dates.sort()

    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    
    index = pd.date_range(startdate, enddate,freq='D')
    df = pd.DataFrame(index = index, columns = ['newCases','newTests'])
    for entry in datalist:
        date = parse_date(entry['date'])
        for column in ['newCases','newTests']:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column]!=None else 0.0
                df.loc[date,column] = value
    df.fillna(0.0, inplace = True)
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

**Click on the button** below to download the most current data, the graphs will be refreshed automatically.

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    global f #mark if the process failed
    try:
        filters = ['areaType=overview' ]

        structure = {
            "date": "date",
            "newCases": "newCasesByPublishDate",
            "newTests":"newTestsByPublishDate"
        }
        api = Cov19API(filters=filters, structure=structure)
        f=False
        return api.get_json() # return data read from the API
    except:
        f=True
        return jsondata 

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    if f:
        apibutton.icon = "unlink"
        description = "Unavailable"
        apibutton.tooltip='fail to download the data'
    else:
        apibutton.icon="check"
        apibutton.disabled=True
        apibutton.tooltip='You have download the data successfully'
        
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
   

    
apibutton=wdg.Button(
    description='Refresh data', 
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

This graph shows a comparison of the **number of new cases** with the **number of tests**.

* You can add or remove tags if you want to see **number of new cases** or the **number of tests** alone.

* You can also change between linear graph and log graph. 

In [7]:
def plot_random_walk(graphcolumns,walk):
    if walk=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(graphcolumns)
    if ncols>0:
        df[list(graphcolumns)].plot(y=list(graphcolumns),logy=logscale)
        plt.show()
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(You can add multiple tags to show more than one category)")
    
    
tags = wdg.TagsInput(
    value=['newCases', 'newTests'],
    allowed_tags=['newCases', 'newTests'],
    allow_duplicates=False,
    description='Type'
)
whichwalk=wdg.Select(
    options=['linear', 'log'],
    description='Scale:',
    value='linear',
    disabled=False
)

controls = wdg.VBox([tags,whichwalk])

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back
    
    
graph=wdg.interactive_output(plot_random_walk, {'graphcolumns':tags,'walk': whichwalk})
    
display(controls, graph)

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*